In [54]:
import numpy as np
import numpy.random as rd
import pandas as pd
from IPython.display import display, Math, HTML

In [55]:
def f(u):
    return np.cos(2 * np.pi * (np.mean(u) - 1/2))

In [56]:
# 1- Monte-Carlo
display(HTML(r"Notice that $I := \int_{[0, 1]^d} f(u) du = \mathbb{E}_{U \sim \mathcal{U}([0, 1]^d)}[f(U)]$. Thus, we can approximate $I$ using Monte Carlo."))

In [57]:
def montecarlo(d, N = 10000):
    X = [f(rd.random(d)) for _ in range(N)]
    return np.mean(X)

In [68]:
d_values = [2, 3, 5, 10, 100]
N_values = [100, 1000, 10000]

data = {N: [montecarlo(d, N) for d in d_values] for N in N_values}

df = pd.DataFrame(data, index = d_values)
df.head()

,100,1000,10000
2,0.374299,0.385124,0.404313
3,0.607797,0.596365,0.562741
5,0.735572,0.724539,0.717093
10,0.852712,0.841630,0.846225
100,0.983933,0.983691,0.983480
